# Open Source convert .dlis to .las v2.0 using dlisio and lasio python packages

## Still a work in progress.  Need to dos:
 - better control over curve metadata from different frames
 - package into a function.

## But this code will handle multiple embedded files within a dlis and multiple frames in a single dlis.  This code will also handle scenarios where values are actually lists of values.

In [ ]:
import os
import pandas as pd
import numpy as np
import lasio
import dlisio
from datetime import datetime

In [ ]:
#filepath = r"...\Volve_Well_logs_pr_WELL\15_9-F-4\02.LWD_EWL\WL_RAW_CAL-DEN-GR-NEU-REMP_MWD_1.DLIS"
filepath = r"....\Volve_Well_logs_pr_WELL\15_9-F-4\02.LWD_EWL\WL_RAW_AAC_MWD_2.DLIS"

In [ ]:
embedded_files = []
origins = []
frames = []
frame_count = 0

## First, you will want to figure out what your index track is called.  In most cases it is TDEP or TIME - but looking at the curve index below you can quickly find out.

In [ ]:
curves_name = []
longs = []
unit = []

with dlisio.load(filepath) as file:
    for d in file:
        for channel in d.channels:
            curves_name.append(channel.name)
            longs.append(channel.long_name)
            unit.append(channel.units)
curve_index = pd.DataFrame(
{'Curve': curves_name,
'Long': longs,
'Unit': unit
})

In [ ]:
curve_index

### If it is TDEP or TIME we will put the index name automatically.

In [ ]:
if "TDEP" in curves_name:
    index_name='TDEP'
elif "TIME" in curves_name:
    index_name='TIME'

## And we need the units of the index tracks for the las header

In [ ]:
unit = curve_index.loc[curve_index["Curve"] == index_name, "Unit"].values[0]

## Second we will read the dlis file for the curves values (from ndarray to pandas), expand out any values that are actually lists, assign the index track, get the metadata, and output the .las files.  One file will be outputted per embedded file per frame.

In [ ]:
with dlisio.load(filepath) as file:
    for d in file:
        curves_L = []
        embedded_files.append(d)
        for origin in d.origin:
            origins.append(origin)
            print(len(origins))
        for fram in d.frames:
            frame_count = frame_count+1
            las = lasio.LASFile()
            fingerprint = fram.fingerprint
            curves = d.curves(fingerprint)
            curves_L.append(curves)
            converted_curves = tuple(curves)
            curves_df = pd.DataFrame.from_records(converted_curves, columns=curves.dtype.names)
            curves_df.head()
            for column in curves_df:
                if curves_df[column].dtype.name == 'object':
                    curves_df = curves_df.assign(**pd.DataFrame(curves_df[column].values.tolist()).add_prefix(column))
                    curves_df = curves_df.drop(columns=[column])
            curves_df = curves_df.set_index(index_name)
            las.set_data(curves_df)
            las.curves[index_name].unit = unit
            las.well.COMP = origin.company
            las.well.WELL = origin.well_name
            las.well.FLD = origin.field_name
            las.well.SRVC = origin.producer_name
            las.well.DATE = origin.creation_time
            las.well.UWI = origin.well_id
            las.well.API = origin.well_id
            las.params['PROD'] = lasio.HeaderItem('PROD', value=origin.product)
            las.params['PROG'] = lasio.HeaderItem('PROG', value=origin.programs)
            las.params['RUN'] = lasio.HeaderItem('RUN', value=origin.run_nr)
            las.params['DESCENT'] = lasio.HeaderItem('DESCENT', value=origin.descent_nr)
            las.params['VERSION'] = lasio.HeaderItem('VERSION', value=origin.version)
            las.params['LINEAGE'] = lasio.HeaderItem('LINEAGE', value="Python-converted from DLIS")
            las.params['ORFILE'] = lasio.HeaderItem('ORFILE', value=filepath)
            las.write('C:\\Users\\aruss\\OneDrive - Equinor\\CDA Data\\DLIS LAS Conversion\\scratch_v2_'+str(frame_count)+'.las', version=2)

## Warnings :  You will get more than one las file as output if there are multiple files in a single .dlis or multiple frames in a single .dlis

In [ ]:
print("embedded_files: "+str(len(embedded_files)))
print("This file has "+str(len(origins))+ " metadata headers.  This code has used the first.")
print("number of frames: "+str(frame_count))